In [2]:
import pandas as pd
import numpy as np
import re
import regex
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from Carga_dataset import *

In [21]:
#Convertimos la columna en string para poder trabajar con el texto
df['Descripcion limpia']= df['descripcion'].astype(str)
df['Descripcion limpia']

id_producto
2792-88-LE21-1        CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...
1704-52-LP21-1        SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...
1003-20-L121-1                                    LEÑA SECA CERTIFICADA
4236-21-L121-1                             MARCADOR BLANCO PARA METALES
4236-21-L121-2                                                  PLOMADA
                                            ...                        
1058141-35-LQ21-1              ALTEPLASA FA 50 MG CAJA X 2 FA [216P008]
1058141-35-LQ21-4           TENECTEPLASA FA 10.000 UI (50 MG) [216P185]
1173455-12-LE21-41               manguera transparente para nivel (mts)
1057461-64-LE21-17                        CINTA ADHESIVA 18 MM X 30 MT 
4309-381-LQ21-1       10-100-214-208-01 Olanzapina tableta liofiliza...
Name: Descripcion limpia, Length: 54849, dtype: object

In [5]:
#Removemos la palabra "no" de nuestro stopword ya que nos puede ser útil
sw = stopwords.words('spanish')
sw.remove("no")

In [19]:
#Definimos nuestra función para quitar las tildes
def sin_tildes(s):
    tildes = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for origen, destino in tildes:
        s = s.replace(origen, destino)
    return s




#Definimos una función para el preprocesamiento de texto

def texto_limpio(texto):
    texto = texto.lower() # convertir en minúsculas
    texto = re.sub(r"[\W\d_]+", " ",texto) # remover caract especiales y números
    texto = sin_tildes(texto) # remove tildes
    texto = texto.split() # tokenizar
    texto = [palabra for palabra in texto if len(palabra) > 3] # eliminar palabras con menos de 3 letras
    texto = [palabra for palabra in texto if palabra not in sw] # stopwords
    texto = " ".join(texto)
    return texto

In [22]:
df['Descripcion limpia'] = df['Descripcion limpia'].apply(lambda texto: texto_limpio(texto)) #Aplicamos la función texto_limpio para limpiar las descripciones
df.head()

,id_licitacion,id_producto,Rubro2,Rubro3,nombre_producto,descripcion,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,...,Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion limpia,Descripcion raiz limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,2792-88-LE21,2792-88-LE21-1,Productos quirúrgicos,"Equipo quirúrgico, accesorios y productos rela...",Equipo quirúrgico de fragmatome retinal vítreo...,CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...,0,0,0,0,...,0,0,0,0,0,0,0,0,camara retinal midriatica especificaciones bas...,cam retinal no midriat especif bas tecnic
1704-52-LP21-1,1704-52-LP21,1704-52-LP21-1,Transporte de correo y carga,Transporte de carga por carretera,Servicios de transporte local en camión,SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...,0,0,0,0,...,0,1,0,0,0,0,0,0,servicio mudanza enseres propios hospital mart...,servici mudanz enser propi uso hospital san ma...
1003-20-L121-1,1003-20-L121,1003-20-L121-1,Chatarra y desechos no metálicos,Desechos no metálicos,Desperdicios o restos de madera,LEÑA SECA CERTIFICADA,0,0,0,0,...,0,0,0,0,0,0,0,0,leña seca certificada,leñ sec certific
4236-21-L121-1,4236-21-L121,4236-21-L121-1,Suministros de oficina,Instrumentos de escritura,Marcadores,MARCADOR BLANCO PARA METALES,0,0,0,0,...,0,0,0,0,0,0,0,0,marcador blanco metales,marcador blanc metal
4236-21-L121-2,4236-21-L121,4236-21-L121-2,Herramientas manuales,Herramientas de medición y trazado,Péndulos,PLOMADA,0,0,0,0,...,0,0,0,0,0,0,0,0,plomada,plom


In [38]:
stemmer=SnowballStemmer("spanish")
#Obtención de texto raíz limpio
def texto_raiz(texto):    
    texto = texto.lower() # convertir en minúsculas
    texto = re.sub(r"[\W\d_]+", " ",texto) # remover caract especiales y números
    texto = sin_tildes(texto) # remove tildes
    texto = texto.split() # tokenizar
    texto = [palabra for palabra in texto if len(palabra) > 3] # eliminar palabras con menos de 3 letras
    texto = [palabra for palabra in texto if palabra not in sw] # stopwords
    texto = [stemmer.stem(palabra) for palabra in texto] #stem
    texto = " ".join(texto)
    
    return texto

In [39]:
df['Descripcion raiz limpia']= df['Descripcion limpia'].apply(lambda texto: texto_raiz(texto)) #Aplicamos la función texto_raiz que nos convierte las palabras en sus raíces
df.head()

,id_licitacion,id_producto,Rubro2,Rubro3,nombre_producto,descripcion,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,...,Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion limpia,Descripcion raiz limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,2792-88-LE21,2792-88-LE21-1,Productos quirúrgicos,"Equipo quirúrgico, accesorios y productos rela...",Equipo quirúrgico de fragmatome retinal vítreo...,CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...,0,0,0,0,...,0,0,0,0,0,0,0,0,camara retinal midriatica especificaciones bas...,cam retinal midriat especif bas tecnic
1704-52-LP21-1,1704-52-LP21,1704-52-LP21-1,Transporte de correo y carga,Transporte de carga por carretera,Servicios de transporte local en camión,SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...,0,0,0,0,...,0,1,0,0,0,0,0,0,servicio mudanza enseres propios hospital mart...,servici mudanz enser propi hospital martin qui...
1003-20-L121-1,1003-20-L121,1003-20-L121-1,Chatarra y desechos no metálicos,Desechos no metálicos,Desperdicios o restos de madera,LEÑA SECA CERTIFICADA,0,0,0,0,...,0,0,0,0,0,0,0,0,leña seca certificada,leñ sec certific
4236-21-L121-1,4236-21-L121,4236-21-L121-1,Suministros de oficina,Instrumentos de escritura,Marcadores,MARCADOR BLANCO PARA METALES,0,0,0,0,...,0,0,0,0,0,0,0,0,marcador blanco metales,marcador blanc metal
4236-21-L121-2,4236-21-L121,4236-21-L121-2,Herramientas manuales,Herramientas de medición y trazado,Péndulos,PLOMADA,0,0,0,0,...,0,0,0,0,0,0,0,0,plomada,plom


In [69]:
#Creamos un nuevo df con los datos deseados para la matriz TD-IDF
df1=df.iloc[:, 6:]
df1.drop(['Descripcion limpia'], axis=1, inplace=True)


In [70]:
#Creamos nuestra bolsa de palabras

count = CountVectorizer()
descripcion = np.array(df1['Descripcion raiz limpia'])
bag = count.fit_transform(descripcion)

In [ ]:
distribucion=nltk.FreqDist(count.vocabulary_) 
lista_frecuencias=distribucion.most_common() #Para ordenar la distribución de la frecuencia de palabras
print(lista_frecuencias)

In [73]:
#Observamos las palabras presentadas en las descripciones limpias
n_palabras= count.vocabulary_
n_palabras

{'cam': 2929,
 'retinal': 15773,
 'midriat': 11890,
 'especif': 6851,
 'bas': 2016,
 'tecnic': 17575,
 'servici': 16535,
 'mudanz': 12299,
 'enser': 6591,
 'propi': 14843,
 'hospital': 9129,
 'martin': 11382,
 'quillot': 15160,
 'ofert': 12991,
 'segun': 16383,
 'anex': 969,
 'leñ': 10668,
 'sec': 16343,
 'certific': 3485,
 'marcador': 11321,
 'blanc': 2390,
 'metal': 11698,
 'plom': 14233,
 'azadon': 1821,
 'mang': 11247,
 'escobillon': 6777,
 'ase': 1471,
 'municipal': 12405,
 'caj': 2841,
 'lapiz': 10447,
 'carpinter': 3193,
 'hoj': 9063,
 'marcosierr': 11332,
 'cint': 3762,
 'papel': 13485,
 'equivalent': 6704,
 'masking': 11401,
 'tap': 17503,
 'pegot': 13723,
 'ancho': 927,
 'carpet': 3190,
 'carton': 3227,
 'pigment': 14041,
 'ofici': 12999,
 'color': 4192,
 'azul': 1840,
 'plastific': 14196,
 'accoclip': 99,
 'archiv': 1310,
 'tamañ': 17484,
 'auc': 1621,
 'cart': 3218,
 'plumon': 14244,
 'permanent': 13887,
 'punt': 15038,
 'redond': 15465,
 'roj': 15968,
 'negr': 12575,
 'ver

In [74]:
len(n_palabras)

19535

In [84]:
from sklearn.feature_extraction.text import TfidfTransformer
#creamos nuestra matriz TF - IDF con el df1
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
#print(tfidf.fit_transform(bag).toarray())

In [85]:
tfidf.fit_transform(bag)

<54849x19535 sparse matrix of type '<class 'numpy.float64'>'
	with 345608 stored elements in Compressed Sparse Row format>

In [82]:
vectorizador = TfidfVectorizer()
 
#Crear una matriz dispersa
cantidades = count.fit_transform(descripcion)
valores = vectorizador.fit_transform(descripcion)
 
print ("--- Palabras ---")
print(vectorizador.get_feature_names())
print ("... Vector de cantidades ...")
print (cantidades)
print ("¨¨¨ Vector de valores ¨¨")
print (valores)



--- Palabras ---
['aac', 'aacc', 'aaguj', 'aalt', 'aasht', 'aatic', 'aavv', 'abacav', 'abadiaisbn', 'abaj', 'abarc', 'abaro', 'abarrot', 'abas', 'abast', 'abastec', 'abaster', 'abat', 'abatacept', 'abatecept', 'abbas', 'abbott', 'abcdin', 'abcidinc', 'abciximab', 'abdom', 'abdominal', 'abduccion', 'abductor', 'abecedari', 'abecedarioi', 'abecidiin', 'abedicin', 'abedul', 'abej', 'abejassofi', 'abejit', 'abelleir', 'aberr', 'abertur', 'abex', 'abiert', 'abirateron', 'abism', 'ablacion', 'abland', 'ablewar', 'abog', 'abomin', 'abon', 'abord', 'abordaj', 'aboton', 'abranquil', 'abras', 'abrasi', 'abrasion', 'abraz', 'abrazader', 'abre', 'abreboc', 'abrefacil', 'abrelat', 'abreti', 'abricot', 'abrig', 'abril', 'abrillant', 'abrir', 'abro', 'abroch', 'absolut', 'absor', 'absorb', 'absorbedor', 'absorbent', 'absorcion', 'abuel', 'abuelit', 'abund', 'aburr', 'acab', 'acabadi', 'acaci', 'academ', 'academi', 'academic', 'acamp', 'acanal', 'acaramel', 'acced', 'acces', 'accesibil', 'accesori', '